In [1]:
dumb_change = 0

smart_change = not dumb_change 

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

import numpy

import cicada
import cicada.active
from cicada.shamir import ShamirBasicProtocolSuite, ShamirProtocolSuite
from cicada.communicator import SocketCommunicator
from copy import deepcopy

def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.active.ActiveProtocol(communicator, threshold=3)

    # Player 0 will provide a secret which is a scalar.
    secret = numpy.array([numpy.pi,1,-2,3]) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}")

    # Create shares for the secret.
    share = protocol.share(src=0, secret=secret, shape=(4,))
    #log.info(f"Player {communicator.rank} share: {share}")

    modulus = 2**64-59
    #log.info(f"Player {communicator.rank} share consistency check: {protocol.check_commit(share)}")
    log.info(f"Player {communicator.rank} share reveal check: {protocol.reveal(share)}")
    log.info(f"Player {communicator.rank} Entering Malicious activity, since we're messing with things in a {['dumb', 'smart'][smart_change]} way,\nthe consistency check should fail in the {['first', 'second'][smart_change]} step.", src=0)
    bad_share = deepcopy(share)
    if protocol.communicator.rank == 3 and dumb_change:
        bad_share[0].storage[0] += 1
    if protocol.communicator.rank == 0 and smart_change:
        bad_share[0].storage[1] += 1
        bad_share[1].storage[1] = (bad_share[1].storage[1] + pow(5, modulus-2, modulus)) % modulus
    try:
        log.info(f"Player {communicator.rank} share consistency check - should be True: {protocol.verify(bad_share)}")
    except cicada.active.ConsistencyError as e:
        print(f'Malicious alteration detected: {e}')
    try:
        log.info(f"Player {communicator.rank} share reveal check: {protocol.reveal(bad_share)}")
    except cicada.active.ConsistencyError as e:
        log.info(f'Malicious alteration detected: {e}')

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Player 0 secret: [ 3.14159265  1.         -2.          3.        ]
INFO:root:Player 1 secret: None
INFO:root:Player 2 secret: None
INFO:root:Player 3 secret: None
INFO:root:Player 4 secret: None
INFO:root:Player 0 share reveal check: [ 3.1415863  1.        -2.         3.       ]
INFO:root:Player 1 share reveal check: [ 3.1415863  1.        -2.         3.       ]
INFO:root:Player 2 share reveal check: [ 3.1415863  1.        -2.         3.       ]
INFO:root:Player 3 share reveal check: [ 3.1415863  1.        -2.         3.       ]
INFO:root:Player 4 share reveal check: [ 3.1415863  1.        -2.         3.       ]
INFO:root:Player 0 Entering Malicious activity, since we're messing with things in a smart way,
the consistency check should fail in the second step.
INFO:root:Player 0 share consistency check - should be True: True
INFO:root:Player 1 share consistency check - should be True: True
INFO:root:Player 2 share consistency check - should be True: True
INFO:root:Player 3 sha

In [3]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.active.ActiveProtocol(communicator, threshold=3)

    # Player 0 will provide a secret which is a scalar.
    secret = numpy.array([numpy.pi,1,-2,3]) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}")

    # Create shares for the secret.
    share = protocol.share(src=0, secret=secret, shape=(4,))
    #log.info(f"Player {communicator.rank} share: {share}")

    modulus = 2**64-59

    double_share = protocol.add(share, share)
    try:
        log.info(f"Player {communicator.rank} share consistency check - should be True: {protocol.verify(double_share)}")
    except cicada.active.ConsistencyError as e:
        print(f'Malicious alteration detected: {e}')
    try:
        log.info(f"Player {communicator.rank} double share reveal check: {protocol.reveal(double_share)}")
    except cicada.active.ConsistencyError as e:
        log.info(f'Malicious alteration detected: {e}')
        
    square_share = protocol.untruncated_multiply(share, share)
    square_share = protocol.truncate(square_share)
    try:
        log.info(f"Player {communicator.rank} share consistency check - should be True: {protocol.verify(square_share)}")
    except cicada.active.ConsistencyError as e:
        print(f'Malicious alteration detected: {e}')
    try:
        log.info(f"Player {communicator.rank} share reveal check: {protocol.reveal(square_share)}")
    except cicada.active.ConsistencyError as e:
        log.info(f'Malicious alteration detected: {e}')

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Player 0 secret: [ 3.14159265  1.         -2.          3.        ]
INFO:root:Player 1 secret: None
INFO:root:Player 2 secret: None
INFO:root:Player 3 secret: None
INFO:root:Player 4 secret: None
INFO:root:Player 0 share consistency check - should be True: True
INFO:root:Player 1 share consistency check - should be True: True
INFO:root:Player 2 share consistency check - should be True: True
INFO:root:Player 3 share consistency check - should be True: True
INFO:root:Player 4 share consistency check - should be True: True
INFO:root:Player 0 double share reveal check: [ 6.28317261  2.         -4.          6.        ]
INFO:root:Player 1 double share reveal check: [ 6.28317261  2.         -4.          6.        ]
INFO:root:Player 2 double share reveal check: [ 6.28317261  2.         -4.          6.        ]
INFO:root:Player 3 double share reveal check: [ 6.28317261  2.         -4.          6.        ]
INFO:root:Player 4 double share reveal check: [ 6.28317261  2.         -4.       

In [4]:
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)
    protocol = cicada.active.ActiveProtocol(communicator, threshold=3)

    # Player 0 will provide a secret which is a scalar.
    secret = numpy.array([numpy.pi,1,-2,3]) if communicator.rank == 0 else None
    log.info(f"Player {communicator.rank} secret: {secret}")

    # Create shares for the secret.
    share = protocol.share(src=0, secret=secret, shape=(4,))
    #log.info(f"Player {communicator.rank} share: {share}")

    modulus = 2**64-59

    square_share = protocol.untruncated_multiply(share, share)
    square_share = protocol.truncate(square_share)
    try:
        log.info(f"Player {communicator.rank} share consistency check - should be True: {protocol.verify(square_share)}")
    except cicada.active.ConsistencyError as e:
        print(f'Malicious alteration detected: {e}')
    try:
        log.info(f"Player {communicator.rank} share reveal check: {protocol.reveal(square_share)}")
    except cicada.active.ConsistencyError as e:
        log.info(f'Malicious alteration detected: {e}')
        
    log.info(f"\n\nFinally we'll mess with the results of the previous computation now, since we're messing with things in a {['dumb', 'smart'][smart_change]} way,\nthe consistency check should fail in the {['first', 'second'][smart_change]} step.", src=0)
    if protocol.communicator.rank == 3 and dumb_change:
        square_share[0].storage[0] += 1
    if protocol.communicator.rank == 0 and smart_change:
        square_share[0].storage[1] += 1
        square_share[1].storage[1] = (square_share[1].storage[1] + pow(5, modulus-2, modulus)) % modulus
    try:
        log.info(f"Player {communicator.rank} share consistency check - should be True: {protocol.verify(square_share)}")
    except cicada.active.ConsistencyError as e:
        print(f'Malicious alteration detected: {e}')
    try:
        log.info(f"Player {communicator.rank} share reveal check: {protocol.reveal(square_share)}")
    except cicada.active.ConsistencyError as e:
        log.info(f'Malicious alteration detected: {e}')

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Player 0 secret: [ 3.14159265  1.         -2.          3.        ]
INFO:root:Player 1 secret: None
INFO:root:Player 2 secret: None
INFO:root:Player 3 secret: None
INFO:root:Player 4 secret: None
INFO:root:Player 0 share consistency check - should be True: True
INFO:root:Player 1 share consistency check - should be True: True
INFO:root:Player 2 share consistency check - should be True: True
INFO:root:Player 3 share consistency check - should be True: True
INFO:root:Player 4 share consistency check - should be True: True
INFO:root:Player 0 share reveal check: [9.86956787 1.         4.         9.        ]
INFO:root:Player 1 share reveal check: [9.86956787 1.         4.         9.        ]
INFO:root:Player 2 share reveal check: [9.86956787 1.         4.         9.        ]
INFO:root:Player 3 share reveal check: [9.86956787 1.         4.         9.        ]
INFO:root:Player 4 share reveal check: [9.86956787 1.         4.         9.        ]
INFO:root:

Finally we'll mess with the 

Thus active protocol exposes similar capabilities to additive or Shamir individually, but allows for honest parties to detect the influences of active malicious adversaries such that their alterations to the shared computation will be detected. The cost to this is roughly 2x the computational and communication cost of either of the protocols individually. 